In [ ]:
import json
import os
import sys
import fnmatch


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict

In [ ]:
df = pd.read_csv('avg_stats.csv')#pd.read_csv('BBGM_League_7_all_seasons_Average_Stats.csv')
df = df[(df.G*df.MP > 48)]
df.shape

In [ ]:
y = df.iloc[:,-18:-17] #45
y = df.iloc[:,45:46] #45

X = df.iloc[:,-15:] # -17:-16
X = X[[_ for _ in X.columns if _ != '+/-' ]]
X.columns,y.columns
#df.columns[4

In [ ]:
_ = plt.hist(y.iloc[:,0],20)

In [ ]:
from sklearn import neural_network
from sklearn import linear_model
from sklearn import preprocessing
from sklearn import feature_selection
from sklearn import  multioutput
from sklearn import ensemble
from sklearn import svm

In [ ]:
fexp = preprocessing.PolynomialFeatures(degree=1,interaction_only=True)
scalerX = preprocessing.RobustScaler()
scalery = preprocessing.StandardScaler()
prescale_X = scalerX.fit_transform(X)
prescale_y = scalery.fit_transform(y)
prescale_X = fexp.fit_transform(prescale_X)

In [ ]:
trials = 1
ts = []
for i in range(trials):
    #clf = neural_network.MLPRegressor((36,5,24,36),'tanh',solver='adam',max_iter=1000)
    #clf = neural_network.MLPRegressor((),'identity',solver='lbfgs',alpha=5e2,tol=1e-9)
    #clf = multioutput.MultiOutputRegressor(linear_model.SGDRegressor(penalty='l2',alpha=5e2,eta0=1e-6,tol=1e-12,max_iter=50,verbose=True))
    clf = multioutput.MultiOutputRegressor(linear_model.ElasticNet(alpha=5e-3,fit_intercept=False))
    #clf = ensemble.ExtraTreesRegressor(8,criterion='mae',max_depth=3,verbose=1)
    #clf = multioutput.MultiOutputRegressor(svm.SVR())
    clf.fit(prescale_X,prescale_y)
    yt = scalery.inverse_transform(clf.predict(prescale_X))
    err = np.linalg.norm(yt-y)
    ts.append((err,clf))

In [ ]:
ts = sorted(ts)[::1] # why not the biggest error
print(ts[0][0])
clf = ts[0][1]

In [ ]:
col_names = X.columns
col_names = fexp.get_feature_names(X.columns)

for i,c in enumerate(y.columns):
    coeffs = clf.estimators_[i].coef_ 
    v = np.argsort(abs(coeffs))[::-1]
    print(c)
    coeffs2 = [(coeffs[i2],col_names[i2]) for i2 in v]
    #for v,n in sorted(coeffs2,reverse=True):
    #    print('{:.2f} * {} + '.format(v,n),end='')
    print('| Var      | Coeff |')
    print('|----------|-------|')
    for v,n in sorted(coeffs2,reverse=True):
        print('|{:10s}|{:.2f}|'.format(n,v/coeffs.max()))
    #for v,n in sorted(coeffs2,reverse=True):
    #    print('\t{:25s}\t{:.2f}'.format(n,v))


In [ ]:
nc_M = []
for n,w in zip(col_names,clf.estimators_[0].coef_):
    if n == '1':
        continue
    w /= (clf.estimators_[0].coef_.sum())*0.01
    w = int(round(w))
    print(w,end=', ')
    nc_M.append(w)
nc_M = np.array(nc_M)

In [ ]:
GEN_YEAR = 2019
with open('auto_roster_{}.json'.format(GEN_YEAR),'r') as fp:
    base = json.load(fp)
players = base['players']

In [ ]:
dkeys = [_.lower() for _ in ['Hgt', 'Stre', 'Spd', 'Jmp', 'Endu', 'Ins', 'Dnk', 'FT', 'fg', 'tp',
       'oIQ', 'dIQ', 'Drb', 'Pss', 'Reb']]
nc = np.array([7,1,6,1,1,1,2,1,1,1,9,3,6,2,1])

In [ ]:
player_df = []
for p in players:
    if not 'name' in p:
        p['name'] = p['firstName'] + ' ' + p['lastName']
    d = {'name':p['name']}
    d.update(p['ratings'][0])
    d['ovr_og'] = sum([d[k]*v for k,v in zip(dkeys,nc)])/nc.sum()
    d['ovr_mn'] = sum([d[k]*v for k,v in zip(dkeys,nc_M)])/nc_M.sum()
    #d['ovr_mnr'] = sum([d[k]*v for k,v in zip(dkeys,nc)])/np.maximum(0,nc3).sum()

    player_df.append(d)

In [ ]:
pdf = pd.DataFrame(player_df).set_index('name')
pdf['diff'] = pdf['ovr_mn'] - pdf['ovr_og']
#pdf['diffr'] = pdf['ovr_mnr'] - pdf['ovr_og']


In [ ]:
pdf.sort_values(['ovr_og'],0,False)


In [ ]:
pdf.sort_values(['ovr_mn'],0,False)


In [ ]:
pdf.sort_values(['diff'],0,False)


In [ ]:
_ = plt.hist(pdf['diff'],40)